# INCEPTION MODULE

# STEM

In [ ]:
def STEM_A(x , device_):
    with tf.device(device_):
        in_ch=x.get_shape()[3]
        out_ch1=32 ; out_ch2=32;out_ch3=64;out_ch4=96;

        c_ksize1=[3,3,in_ch , out_ch1]
        c_ksize2=[3,3,out_ch1 , out_ch2]
        c_ksize3=[3,3,out_ch2 , out_ch3]
        c_ksize4=[3,3,out_ch3 , out_ch4]
        w_conv1 , b_conv1 =make_weights_biases('STEM_A' , 'W1' , c_ksize1 ,device_name = '/gpu:0')
        w_conv2 , b_conv2= make_weights_biases('STEM_A' , 'W2' , c_ksize2 ,device_name = '/gpu:0')
        w_conv3 , b_conv3= make_weights_biases('STEM_A' , 'W3' , c_ksize3 ,device_name = '/gpu:0')
        w_conv4 , b_conv4= make_weights_biases('STEM_A' , 'W4' , c_ksize4 ,device_name = '/gpu:0')

        c_strides1=[1,2,2,1]
        c_strides2=[1,1,1,1]
        c_strides3=[1,1,1,1]
        c_strides4=[1,2,2,1]

        c_pooling1='VALID'
        c_pooling2='VALID'
        c_pooling3='SAME'
        c_pooling4='VALID'

        b_p_ksize4=[1,3,3,1]
        b_p_strides4=[1,2,2,1]
        b_p_padding4 ='VALID'



        layer1=tf.nn.conv2d(    x ,      w_conv1, c_strides1, c_pooling1  )+b_conv1
        layer1=tf.nn.relu(layer1)
        layer2=tf.nn.conv2d(    layer1 , w_conv2, c_strides2, c_pooling2  )+b_conv2
        layer2=tf.nn.relu(layer2)
        layer3=tf.nn.conv2d(    layer2 , w_conv3, c_strides3, c_pooling3  )+b_conv3
        layer3=tf.nn.relu(layer3)
        layer4=tf.nn.conv2d(    layer3 , w_conv4, c_strides4, c_pooling4  )+b_conv4
        layer4=tf.nn.relu(layer4)


        b_layer4=tf.nn.max_pool(layer3 , b_p_ksize4, b_p_strides4, b_p_padding4 ) #b is branch
        b_layer4=tf.nn.relu(b_layer4)
        print layer1
        print layer2
        print layer3
        print layer4
        print b_layer4


        concat_layer=tf.concat(3 , [layer4 , b_layer4])
        ret_layer=concat_layer
        print concat_layer
        return ret_layer

In [ ]:
def STEM_B( x , device_):
    with tf.device(device_):
        in_ch=x.get_shape()[3]


        #########################################################################
        out_ch1=64 ; out_ch2=64;out_ch3=64;out_ch4=96;
        ##############################right side##################################
        c_ksize1=[1,1,in_ch , out_ch1]
        c_ksize2=[7,1,out_ch1 , out_ch2]
        c_ksize3=[1,7,out_ch2 , out_ch3]
        c_ksize4=[3,3,out_ch3 , out_ch4]

        w_conv1 , b_conv1 =make_weights_biases('STEM_B' , 'W1' , c_ksize1 ,device_name = device_)
        w_conv2 , b_conv2= make_weights_biases('STEM_B' , 'W2' , c_ksize2 ,device_name = device_)
        w_conv3 , b_conv3= make_weights_biases('STEM_B' , 'W3' , c_ksize3 ,device_name = device_)
        w_conv4 , b_conv4= make_weights_biases('STEM_B' , 'W4' , c_ksize4 ,device_name = device_)

        c_strides1=[1,1,1,1]
        c_strides2=[1,1,1,1]
        c_strides3=[1,1,1,1]
        c_strides4=[1,1,1,1]

        c_pooling1='SAME'
        c_pooling2='SAME'
        c_pooling3='SAME'
        c_pooling4='VALID'
        
        layer1 = tf.nn.conv2d(x ,      w_conv1 , c_strides1 , c_pooling1)+b_conv1
        layer1 = tf.nn.relu(layer1)
        layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2)+b_conv2
        layer2 = tf.nn.relu(layer2)
        layer3 = tf.nn.conv2d(layer2 , w_conv3 , c_strides3 , c_pooling3)+b_conv3
        layer3 = tf.nn.relu(layer3)
        layer4 = tf.nn.conv2d(layer3 , w_conv4 , c_strides4 , c_pooling4)+b_conv4
        layer4 = tf.nn.relu(layer4)

        ##############################left side##################################
        b_out_ch1=64 ; b_out_ch2=96;    
        #########################################################################
        b_c_ksize1=[1,1,in_ch , b_out_ch1]
        b_c_ksize2=[3,3,b_out_ch1 , b_out_ch2]
        b_w_conv1 , b_b_conv1 =make_weights_biases('STEM_B' , 'b_W1' , b_c_ksize1 ,device_name = device_)
        b_w_conv2 , b_b_conv2= make_weights_biases('STEM_B' , 'b_W2' , b_c_ksize2 ,device_name = device_)

        b_c_strides1=[1,1,1,1]
        b_c_strides2=[1,1,1,1]

        b_c_pooling1='SAME'
        b_c_pooling2='VALID'
        
        b_layer1 =tf.nn.conv2d(    x ,      b_w_conv1, b_c_strides1, b_c_pooling1  ) + b_b_conv1
        b_layer1 = tf.nn.relu(layer1)
        b_layer2=tf.nn.conv2d(  b_layer1 , b_w_conv2, b_c_strides2, b_c_pooling2  ) + b_b_conv2

        ##############################concatenate layers###########################
        concat_layer=tf.concat(3 , [layer4 , b_layer2])
        ret_layer=concat_layer
    return ret_layer

In [ ]:
def STEM_C( x , device_ ):
    with tf.device(device_):
        in_ch=x.get_shape()[3]
        #########################################################################

        out_ch = 192


        c_ksize=[3,3,in_ch,out_ch]
        w_conv , b_conv =make_weights_biases('STEM_C' , 'W1' , c_ksize ,device_name = device_)
        c_strides=[1,1,1,1]
        c_pooling='SAME'

        layer = tf.nn.conv2d(x , w_conv   , c_strides   , c_pooling)+b_conv
        layer = tf.nn.relu(layer)
        
        #########################################################################
        
        b_p_ksize=[1,2,2,1]
        b_p_strides=[1,1,1,1]
        b_p_pooling='SAME'

        b_layer = tf.nn.max_pool(x , b_p_ksize , b_p_strides , b_p_pooling)
        b_layer = tf.nn.relu(b_layer)
        #########################################################################

        print layer
        print b_layer

        concat_layer=tf.concat(3,[layer , b_layer])
        ret_layer = concat_layer
    return ret_layer

# MODULE A

In [ ]:
def MODULE_A(x , device_):

    """
    input X shape is [n_batch , row , col, out_ch]
    35 x 35 grid
    """

    #################################################################################
    in_ch=x.get_shape()[3]
    out_ch1=64 ; out_ch2= 96;
    c_ksize1 = [1 , 1 ,in_ch , out_ch1 ]
    c_ksize2 = [3 , 3 ,out_ch1 , out_ch2 ]
    w_conv1 , b_conv1 = make_weights_biases ('INCEPTION_MODULE_A' , 'W1' , c_ksize1 ,device_)
    w_conv2 , b_conv2 = make_weights_biases('INCEPTION_MODUEL_A' ,'W1' , c_ksize2 ,device_)

    c_strides1=[1,1,1,1]
    c_strides2=[1,1,1,1]
    
    c_pooling1='SAME'
    c_pooling2='SAME'
    layer1 = tf.nn.conv2d(x , w_conv1 , c_strides1 ,c_pooling1 )
    layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 ,c_pooling2 )

    #################################################################################                     

    b1_p_ksize   =[1,2,2,1]
    b1_p_strides =[1,1,1,1]
    b1_p_pooling='SAME'


    b1_out_ch = 96;
    b1_c_ksize  =[1,1, in_ch , b1_out_ch]
    b1_c_strides=[1,1,1,1]
    b1_c_pooling ='SAME'

    b1_w_conv , b1_b_conv =make_weights_biases('INCEPTION_MODULE_A','b1_W',b1_c_ksize , device_)


    b1_layer1=tf.nn.avg_pool(x, b1_p_ksize , b1_p_strides,b1_p_pooling)
    b1_layer1=tf.nn.relu(b1_layer1)
    b1_layer2=tf.nn.conv2d(b1_layer1 , b1_w_conv , b1_c_strides , b1_c_pooling)+b1_b_conv
    b1_layer2=tf.nn.relu(b1_layer2)

    #################################################################################                     

    b2_out_ch=96
    b2_c_ksize = [1,1,in_ch , b2_out_ch ]
    b2_w_conv , b2_b_conv = make_weights_biases('INCEPTION_MODULE_A','b2_W',b2_c_ksize , device_)
    b2_c_strides=[1,1,1,1]
    b2_c_pooling='SAME'

    b2_layer = tf.nn.conv2d( x, b2_w_conv , b2_c_strides ,b2_c_pooling ) +b2_b_conv 
    b2_layer = tf.nn.relu(b2_layer)



    #################################################################################                     


    b3_out_ch1=64;b3_out_ch2=96;b3_out_ch3=96;
    b3_c_ksize1 =[1,1,in_ch , b3_out_ch1]
    b3_c_ksize2 =[3,3,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3 =[3,3,b3_out_ch2 , b3_out_ch3]
    b3_w_conv1 , b3_b_conv1 = make_weights_biases('INCEPTION_MODULE_A','b3_W1',b3_c_ksize1 , device_)
    b3_w_conv2 , b3_b_conv2 = make_weights_biases('INCEPTION_MODULE_A','b3_W2',b3_c_ksize2 , device_)
    b3_w_conv3 , b3_b_conv3 = make_weights_biases('INCEPTION_MODULE_A','b3_W3',b3_c_ksize3 , device_)
    b3_c_strides1=[1,1,1,1]
    b3_c_strides2=[1,1,1,1]
    b3_c_strides3=[1,1,1,1]
    b3_c_pooling1='SAME'
    b3_c_pooling2='SAME'
    b3_c_pooling3='SAME'

    b3_layer1 = tf.nn.conv2d(x , b3_w_conv1 , b3_c_strides1 ,b3_c_pooling1)+b3_b_conv1 
    b3_layer1 = tf.nn.relu(b3_layer1)
    b3_layer2 = tf.nn.conv2d(b3_layer1 , b3_w_conv2 , b3_c_strides2 ,b3_c_pooling2)+b3_b_conv2 
    b3_layer2 = tf.nn.relu(b3_layer2)
    b3_layer3 = tf.nn.conv2d(b3_layer2 , b3_w_conv3 , b3_c_strides3 ,b3_c_pooling3)+b3_b_conv3 
    b3_layer3 = tf.nn.relu(b3_layer3)

    #################################################################################                     
    concat_A=tf.concat(3,[layer2 , b1_layer2])
    concat_B=tf.concat(3,[concat_A , b2_layer])
    concat_C=tf.concat(3,[concat_B , b3_layer3])

    return concat_C

    #################################################################################                     





# MODUEL B

In [ ]:
def MODULE_B(x , device):

    """
    for 17 X 17 grid 
    """
    in_ch =x.get_shape()[3]
    out_ch1 =192 ; out_ch2=224; out_ch3 =256
    c_ksize1 = [1,1, in_ch  ,out_ch1]
    c_ksize2 = [1,7, out_ch1,out_ch2]
    c_ksize3 = [7,1, out_ch2,out_ch3]

    w_conv1 , b_conv1 =make_weights_biases('INCEPTION_MODULE_B' ,'W1' ,  c_ksize1 , device)
    w_conv2 , b_conv2 =make_weights_biases('INCEPTION_MODULE_B' ,'W2' ,  c_ksize2 , device)
    w_conv3 , b_conv3 =make_weights_biases('INCEPTION_MODULE_B' ,'W3' ,  c_ksize3 , device)
    
    c_strides1 =[1,1,1,1]
    c_strides2 =[1,1,1,1]
    c_strides3 =[1,1,1,1]
    
    c_pooling1 ='SAME'
    c_pooling2 ='SAME'
    c_pooling3 ='SAME'


    layer1 = tf.nn.conv2d(x, w_conv1,c_strides1 , c_pooling1 ) +b_conv1
    layer1 = tf.nn.relu(layer1)
    layer2 = tf.nn.conv2d(layer1, w_conv2,c_strides2 , c_pooling2 ) +b_conv2
    layer2 = tf.nn.relu(layer2)
    layer3 = tf.nn.conv2d(layer2, w_conv3,c_strides3 , c_pooling3 ) +b_conv3
    layer3 = tf.nn.relu(layer3)


    ################################################################################# 


    b1_p_ksize=[1,2,2,1]
    b1_p_strides=[1,1,1,1]
    b1_p_pooling ='SAME'

    b1_out_ch = 128
    b1_c_ksize=[1,1,in_ch,b1_out_ch]
    b1_w_conv , b1_b_conv = make_weights_biases('INCEPTION_MODULE_B','b1_W1',b1_c_ksize, device )
    b1_c_strides=[1,1,1,1]
    b1_c_pooling='SAME'


    b1_layer1=tf.nn.avg_pool(x , b1_p_ksize , b1_p_strides , b1_p_pooling )
    b1_layer1=tf.nn.relu(b1_layer1)
    b1_layer2 =tf.nn.conv2d(b1_layer1 ,b1_w_conv , b1_c_strides, b1_c_pooling )+b1_b_conv 
    b1_layer2 =tf.nn.relu(b1_layer2)

    ################################################################################# 
    b2_out_ch=384
    b2_c_ksize=[1,1,in_ch,b2_out_ch]
    b2_w_conv , b2_b_conv =make_weights_biases('INCEPTION_MODULE_B','b2_W1' ,b2_c_ksize , device)
    b2_c_strides=[1,1,1,1]
    b2_c_pooling='SAME'
    b2_layer = tf.nn.conv2d(x,b2_w_conv , b2_c_strides , b2_c_pooling)+b2_b_conv
    b2_layer = tf.nn.relu(b2_layer)
    ################################################################################# 


    b3_out_ch1=192; b3_out_ch2 =192; b3_out_ch3=224 ; b3_out_ch4=224 ; b3_out_ch5 =256
    b3_c_ksize1=[1,1,in_ch,b3_out_ch1]
    b3_c_ksize2=[1,7,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3=[7,1,b3_out_ch2 , b3_out_ch3]
    b3_c_ksize4=[1,7,b3_out_ch3 , b3_out_ch4]
    b3_c_ksize5=[7,1,b3_out_ch4 , b3_out_ch5]

    b3_w_conv1 , b3_b_conv1=make_weights_biases('INCEPTION_MODULE_B','b3_W1',b3_c_ksize1, device)
    b3_w_conv2 , b3_b_conv2=make_weights_biases('INCEPTION_MODULE_B','b3_W2',b3_c_ksize2, device)
    b3_w_conv3 , b3_b_conv3=make_weights_biases('INCEPTION_MODULE_B','b3_W3',b3_c_ksize3, device)
    b3_w_conv4 , b3_b_conv4=make_weights_biases('INCEPTION_MODULE_B','b3_W4',b3_c_ksize4, device)
    b3_w_conv5 , b3_b_conv5=make_weights_biases('INCEPTION_MODULE_B','b3_W5',b3_c_ksize5, device)


    b3_layer1 = tf.nn.conv2d(x,b3_w_conv1 , b2_c_strides , b2_c_pooling)+b3_b_conv1
    b3_layer1 = tf.nn.relu(b3_layer1)
    b3_layer2 = tf.nn.conv2d(b3_layer1,b3_w_conv2 , b2_c_strides , b2_c_pooling)+b3_b_conv2
    b3_layer2 = tf.nn.relu(b3_layer2)
    b3_layer3 = tf.nn.conv2d(b3_layer2,b3_w_conv3 , b2_c_strides , b2_c_pooling)+b3_b_conv3
    b3_layer3 = tf.nn.relu(b3_layer3)
    b3_layer4 = tf.nn.conv2d(b3_layer3,b3_w_conv4 , b2_c_strides , b2_c_pooling)+b3_b_conv4
    b3_layer4 = tf.nn.relu(b3_layer4)
    b3_layer5 = tf.nn.conv2d(b3_layer4,b3_w_conv5 , b2_c_strides , b2_c_pooling)+b3_b_conv5
    b3_layer5 = tf.nn.relu(b3_layer5)

    ################################################################################# 


    layerA=tf.concat(3, [layer3 , b1_layer2] )
    layerB=tf.concat(3, [layerA , b2_layer] )
    layerC=tf.concat(3, [layerB , b3_layer5] )

    print layerC
    return layerC


# MODULE C

In [ ]:
def MODULE_C(x , device):

    """
    for 8 X 8 grid modules
    """
    print x.get_shape()
    in_ch=x.get_shape()[3]
    
    out_ch1 = 384 ; out_ch2_a =256 ;out_ch2_b = 256
    c_ksize1 = [1,1,in_ch , out_ch1]
    c_ksize2_a = [1,3,out_ch1 , out_ch2_a]
    c_ksize2_b = [3,1,out_ch2_a , out_ch2_b]
    w_conv1 ,b_conv1=make_weights_biases('INCEPTION_MODULE_C','W1', c_ksize1 ,device)
    w_conv2_a ,b_conv2_a=make_weights_biases('INCEPTION_MODULE_C','W2_a', c_ksize2_a ,device)
    w_conv2_b ,b_conv2_b=make_weights_biases('INCEPTION_MODULE_C','W2_b', c_ksize2_b ,device)
    c_strides1=[1,1,1,1]
    c_strides2_a=[1,1,1,1]
    c_strides2_b=[1,1,1,1]
    c_pooling1='SAME'
    c_pooling2_a='SAME'
    c_pooling2_b='SAME'

    layer1 = tf.nn.conv2d(x, w_conv1 ,c_strides1 ,c_pooling1)+b_conv1
    layer1=tf.nn.relu(layer1)
    layer2_a = tf.nn.conv2d(layer1,  w_conv2_a ,c_strides2_a ,c_pooling2_a )+b_conv2_a
    layer2_a = tf.nn.relu(layer2_a)
    layer2_b = tf.nn.conv2d(layer1  , w_conv2_a ,c_strides2_a ,c_pooling2_b)+b_conv2_b
    layer2_b = tf.nn.relu(layer2_b)
    ################################################################################# 


    b1_p_ksize=[1,2,2,1]
    b1_p_strides=[1,1,1,1]
    b1_p_pooling='SAME'

    b1_out_ch =256
    b1_c_ksize=[1,1,in_ch , b1_out_ch]
    b1_w_conv , b1_b_conv= make_weights_biases('INCEPTION_MODULE_C' , 'b1_W1' , b1_c_ksize , device)
    b1_c_pooling = 'SAME';b1_c_strides = [1,1,1,1];
    b1_layer1 = tf.nn.avg_pool(x , b1_p_ksize , b1_p_strides , b1_p_pooling)
    b1_layer2 = tf.nn.conv2d(b1_layer1 , b1_w_conv , b1_c_strides  ,b1_c_pooling)+b1_b_conv
    b1_layer2 = tf.nn.relu(b1_layer2)
    ################################################################################# 
    b2_out_ch=256
    b2_c_ksize=[1,1,in_ch,b2_out_ch]
    b2_w_conv , b2_b_conv= make_weights_biases('INCEPTION_MODULE_C' , 'b2_W' , b2_c_ksize , device)
    b2_c_pooling = 'SAME';b2_c_strides=[1,1,1,1];
    b2_layer = tf.nn.conv2d(x , b2_w_conv , b2_c_strides  ,b2_c_pooling)+b2_b_conv
    b2_layer= tf.nn.relu(b2_layer)
   
    ################################################################################# 


    b3_out_ch1=384;b3_out_ch2=448;b3_out_ch3=512;b3_out_ch4_a=256;b3_out_ch4_b=256
    b3_c_ksize1 =[1,1,in_ch , b3_out_ch1]
    b3_c_ksize2 =[1,3,b3_out_ch1 , b3_out_ch2]
    b3_c_ksize3 =[3,1,b3_out_ch2 , b3_out_ch3]
    b3_c_ksize4_a =[3,1,b3_out_ch3 , b3_out_ch4_a]
    b3_c_ksize4_b =[1,3,b3_out_ch3, b3_out_ch4_b]

    b3_w_conv1 , b3_b_conv1 = make_weights_biases('INCEPTION_MODULE_C','b3_W1',b3_c_ksize1 , device)
    b3_w_conv2 , b3_b_conv2 = make_weights_biases('INCEPTION_MODULE_C','b3_W2',b3_c_ksize2 , device)
    b3_w_conv3 , b3_b_conv3 = make_weights_biases('INCEPTION_MODULE_C','b3_W3',b3_c_ksize3 , device)
    b3_w_conv4_a , b3_b_conv4_a = make_weights_biases('INCEPTION_MODULE_C','b3_W4_a',b3_c_ksize4_a , device)
    b3_w_conv4_b , b3_b_conv4_b = make_weights_biases('INCEPTION_MODULE_C','b3_W4_b',b3_c_ksize4_b , device)

    b3_c_strides1=[1,1,1,1]
    b3_c_strides2=[1,1,1,1]
    b3_c_strides3=[1,1,1,1]
    b3_c_strides4_a=[1,1,1,1]
    b3_c_strides4_b=[1,1,1,1]

    b3_c_pooling1='SAME'
    b3_c_pooling2='SAME'
    b3_c_pooling3='SAME'
    b3_c_pooling4_a='SAME'
    b3_c_pooling4_b='SAME'

    b3_laer1 = tf.nn.conv2d(x , b3_w_conv1 , b3_c_strides1 ,b3_c_pooling1)+b3_b_conv1 
    b3_layer1 = tf.nn.relu(b3_laer1)
    b3_layer2 = tf.nn.conv2d(b3_laer1 , b3_w_conv2 , b3_c_strides2 ,b3_c_pooling2)+b3_b_conv2 
    b3_layer2 = tf.nn.relu(b3_layer2)
    b3_layer3 = tf.nn.conv2d(b3_layer2 , b3_w_conv3 , b3_c_strides3 ,b3_c_pooling3)+b3_b_conv3 
    b3_layer3 = tf.nn.relu(b3_layer3)

    b3_layer4_a = tf.nn.conv2d(b3_layer3 , b3_w_conv4_a , b3_c_strides4_a ,b3_c_pooling4_a)+b3_b_conv4_a 
    b3_layer4_a = tf.nn.relu(b3_layer4_a)

    b3_layer4_b = tf.nn.conv2d(b3_layer3 , b3_w_conv4_b , b3_c_strides4_b ,b3_c_pooling4_b)+b3_b_conv4_b 
    b3_layer4_b = tf.nn.relu(b3_layer4_b)


    ################################################################################# 

    layerA = tf.concat(3 , [layer2_a, layer2_b])
    layerB = tf.concat(3 , [layerA  , b1_layer2])
    layerC = tf.concat(3 , [layerB  , b2_layer])
    layerD = tf.concat(3 , [layerC  , b3_layer4_a])
    layerE = tf.concat(3 , [layerD  , b3_layer4_b])

    return layerE

# REDUCTION A

In [ ]:
def REDUCTION_A(x , device):
    ####################################################################################
   
    """
    usage:
    x shape =[ n_batch , row , col , ch] 
    """
    print x.get_shape()
    in_ch=x.get_shape()[3]
    out_ch = 384
    c_ksize= [3,3,in_ch , out_ch]
    w_conv , b_conv =make_weights_biases('INCEPTION_REDUCTION_A','W1',c_ksize,device)
    c_strides=[1,2,2,1]
    c_pooling='VALID'
    layer=tf.nn.conv2d(x,w_conv,c_strides , c_pooling)+b_conv
    layer=tf.nn.relu(layer)

    ####################################################################################
   
    b1_p_ksize=[1,3,3,1]
    b1_p_strides=[1,2,2,1]
    b1_p_pooling='VALID'
    b1_layer = tf.nn.max_pool(x,b1_p_ksize , b1_p_strides , b1_p_pooling)
    ####################################################################################
   
    b2_out_ch1 = 192; b2_out_ch2=288 ;b2_out_ch3 = 256;
    b2_c_ksize1=[1,1,in_ch,b2_out_ch1]
    b2_c_ksize2=[3,3,b2_out_ch1,b2_out_ch2]
    b2_c_ksize3=[3,3,b2_out_ch2,b2_out_ch3]
    b2_w_conv1 , b2_b_conv1 = make_weights_biases('INCEPTION_REDUCTION_A' ,'b2_W1' , b2_c_ksize1 , device)
    b2_w_conv2 , b2_b_conv2 = make_weights_biases('INCEPTION_REDUCTION_A' ,'b2_W2' , b2_c_ksize2 , device)
    b2_w_conv3 , b2_b_conv3 = make_weights_biases('INCEPTION_REDUCTION_A' ,'b2_W3' , b2_c_ksize3 , device)
    
    b2_c_strides1=[1,1,1,1]
    b2_c_strides2=[1,1,1,1]
    b2_c_strides3=[1,2,2,1]
    b2_c_pooling1='SAME'
    b2_c_pooling2='SAME'
    b2_c_pooling3='VALID'
    
    b2_layer1 = tf.nn.conv2d(x        , b2_w_conv1 ,b2_c_strides1 ,b2_c_pooling1)+b2_b_conv1
    b2_layer1 = tf.nn.relu(b2_layer1)
    b2_layer2 = tf.nn.conv2d(b2_layer1, b2_w_conv2 ,b2_c_strides2 ,b2_c_pooling2)+b2_b_conv2
    b2_layer2 = tf.nn.relu(b2_layer2)
    b2_layer3 = tf.nn.conv2d(b2_layer2, b2_w_conv3 ,b2_c_strides3 ,b2_c_pooling3)+b2_b_conv3
    b2_layer3 = tf.nn.relu(b2_layer3)
    
    print layer.get_shape()
    print b1_layer.get_shape()
    print b2_layer3.get_shape()
    ####################################################################################
   
    layerA=tf.concat(3 ,[layer ,b1_layer ])
    layerB=tf.concat(3 ,[layerA ,b2_layer3])
    
    
    
    return layerB
    

# REDUCTION B

In [ ]:
def REDUCTION_B(x , device):
    """
    
    """
    
    ####################################################################################
    in_ch=x.get_shape()[3]
    out_ch1=192 ; out_ch2=192;

    c_ksize1 =[1,1,in_ch , out_ch1]
    c_ksize2 =[3,3,out_ch1,out_ch2]
    w_conv1, b_conv1 = make_weights_biases('INCEPTION_REDUCTION_B','W1',c_ksize1,device)
    w_conv2, b_conv2 = make_weights_biases('INCEPTION_REDUCTION_B','W2',c_ksize2,device)
    
    c_strides1=[1,1,1,1]
    c_strides2=[1,2,2,1]
    
    c_pooling1='SAME'
    c_pooling2='VALID'
    
    layer1 = tf.nn.conv2d(x,w_conv1 , c_strides1, c_pooling1)
    layer2 = tf.nn.conv2d(layer1 , w_conv2 , c_strides2 , c_pooling2)    
    
    b1_p_ksize=[1,3,3,1]
    b1_p_strides=[1,2,2,1]
    b1_p_pooling='VALID'
    
    b1_layer1=tf.nn.max_pool(x ,b1_p_ksize , b1_p_strides , b1_p_pooling)
    
    ####################################################################################
    b2_out_ch1=256 ; b2_out_ch2 = 256 ; b2_out_ch3=320 ; b2_out_ch4=320;
    b2_p_ksize1 =[1,1,in_ch , b2_out_ch1]
    b2_p_ksize2 =[1,7,b2_out_ch1 , b2_out_ch2]
    b2_p_ksize3 =[7,1,b2_out_ch2 , b2_out_ch3]
    b2_p_ksize4 =[3,3,b2_out_ch3 , b2_out_ch4]
    
    b2_w_conv1, b2_b_conv1 = make_weights_biases('INCEPTION_REDUCTION_B','b2_W1' ,b2_c_ksize1,device)
    b2_w_conv2, b2_b_conv2 = make_weights_biases('INCEPTION_REDUCTION_B','b2_W2' ,b2_c_ksize2,device)
    b2_w_conv3, b2_b_conv3 = make_weights_biases('INCEPTION_REDUCTION_B','b2_W3' ,b2_c_ksize3,device)
    b2_w_conv4, b2_b_conv4 = make_weights_biases('INCEPTION_REDUCTION_B','b2_W4' ,b2_c_ksize4,device)
    
    b2_c_strides1=[1,1,1,1]
    b2_c_strides2=[1,1,1,1]
    b2_c_strides3=[1,1,1,1]
    b2_c_strides4=[1,1,1,1]
    
    b2_c_pooling1= 'SAME'
    b2_c_pooling2= 'SAME'
    b2_c_pooling3 = 'SAME'
    b2_c_pooling4 = 'VALID'
    
    b2_layer1 = tf.nn.conv2d(x, b2_w_conv1 , b2_c_strides1 , b2_c_pooling1 )+b2_b_conv1
    b2_layer2 = tf.nn.conv2d(b2_layer1, b2_w_conv2 , b2_c_strides2 , b2_c_pooling2 )+b2_b_conv2
    b2_layer3 = tf.nn.conv2d(b2_layer2, b2_w_conv3 , b2_c_strides3 , b2_c_pooling3 )+b2_b_conv3
    b2_layer4 = tf.nn.conv2d(b2_layer3, b2_w_conv4 , b2_c_strides4 , b2_c_pooling4 )+b2_b_conv4
    
    ####################################################################################